
# Stock Analysis using OpenAI Functions and LangChain Multi Agent

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*

**Diclaimer**: This is only a demo notebook of stocks analysis. The recommendations or advise may not be used for actual invesment decision.

In [1]:
!pip install -q yfinance langchain langchainhub langsmith httpx==0.27.2 langchain_groq langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 13.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.14.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [2]:
import openai
import json
import os
import numpy as np
import yfinance as yf
from getpass import getpass
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

## Defining a custom method for obtaining stats of a stock

**yfinance** is an open source library to access the financial data available on Yahoo Finance, which offers an excellent range of market data on stocks, bonds and currencies along with market news, reports and analysis.

- Define a function which returns total gains, average and standard deviation of daily prices changes of stocks for a specified number of months

In [14]:
@tool
def get_stock_stats(ticker: str, month:str ):

    '''
        ticker: the stock ticker name
        month: number of month for which the stock trading data to used e.g. 1 or 2 or 3 etc.
        input string ticker is The correct ticker symbol for the stock in Nation Stock Exchange (NSE) India.
    '''

    # Get ticker name correctly
    msft = yf.Ticker(ticker.split(".")[0] + ".NS")

    # get historical market data
    hist = msft.history(period = month + "mo" )

    # Compute the market data
    hist['daily_changes']  = (hist['Close'] - hist['Open']) * 100 / hist['Open']

    # Compute different statistics
    total_gain = (hist.iloc[-1]["Close"] - hist.iloc[0]["Open"] ) * 100 / hist.iloc[0]["Open"]
    avg_daily_changes = np.mean(hist['daily_changes'])
    std_daily_changes = np.std(hist['daily_changes'])

    stock_stats = {'total_gain_in_percentage': round(total_gain, 3),
                   'average_daily_changes_in_percentage': round(avg_daily_changes, 3),
                   'std_daily_changes_in_percentage': round(std_daily_changes, 3)}

    return json.dumps(stock_stats)

In [18]:
get_stock_stats.invoke({"ticker": "TATAPOWER", "month": "4" })

'{"total_gain_in_percentage": 11.229, "average_daily_changes_in_percentage": 0.046, "std_daily_changes_in_percentage": 1.96}'

## Configuring the function as a tool

## Configuring tool as OpenAI function

In [6]:
#os.environ["OPENAI_API_KEY"] = getpass('OPENAI_API_KEY')
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [20]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_retries=2,
)

In [21]:
tools = [get_stock_stats]

In [22]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)

In [23]:
resp = agent_executor.invoke({"input": "How to find out if a stock is highly volatile?"})
print(resp)



> Entering new AgentExecutor chain...

Invoking: `get_stock_stats` with `{'ticker': 'INFY', 'month': '3'}`
responded: To determine if a stock is highly volatile, you can use the 'get_stock_stats' function to retrieve the stock's trading data for a specific month. Then, you can analyze the data to calculate the stock's volatility using metrics such as standard deviation or beta. 



{"total_gain_in_percentage": -2.511, "average_daily_changes_in_percentage": -2.511, "std_daily_changes_in_percentage": 0.0}
Invoking: `get_stock_stats` with `{'ticker': 'INFY', 'month': '3'}`
responded: To determine if a stock is highly volatile, you can use the 'get_stock_stats' function to retrieve the stock's trading data for a specific month. The function returns the total gain in percentage, average daily changes in percentage, and standard deviation of daily changes in percentage.

A stock is considered highly volatile if its standard deviation of daily changes in percentage is high. You can use this

In [24]:
resp = agent_executor.invoke({"input": "What is total return for the stock tatapower in last one month?"})
print(resp)



> Entering new AgentExecutor chain...

Invoking: `get_stock_stats` with `{'ticker': 'TATAPOWER', 'month': '1'}`


{"total_gain_in_percentage": 3.813, "average_daily_changes_in_percentage": 0.39, "std_daily_changes_in_percentage": 1.869}The total return for the stock TATAPOWER in the last one month is 3.813%. The average daily change is 0.39% with a standard deviation of 1.869%.

> Finished chain.
{'input': 'What is total return for the stock tatapower in last one month?', 'output': 'The total return for the stock TATAPOWER in the last one month is 3.813%. The average daily change is 0.39% with a standard deviation of 1.869%.'}


## OpenAI Multi Function Agent

In [25]:
response = agent_executor.invoke( {"input":
    "Between tatapower and adanipower in NSE India, which stock had observerd higher volatility in last one month?"}
)



> Entering new AgentExecutor chain...

Invoking: `get_stock_stats` with `{'ticker': 'TATAPOWER', 'month': '1'}`


{"total_gain_in_percentage": 3.813, "average_daily_changes_in_percentage": 0.39, "std_daily_changes_in_percentage": 1.869}
Invoking: `get_stock_stats` with `{'ticker': 'ADANIPOWER', 'month': '1'}`


{"total_gain_in_percentage": 2.424, "average_daily_changes_in_percentage": -0.114, "std_daily_changes_in_percentage": 2.536}Based on the stock trading data for the last one month, TATAPOWER had an average daily change of 0.39% and a standard deviation of daily changes of 1.869%, while ADANIPOWER had an average daily change of -0.114% and a standard deviation of daily changes of 2.536%. 

Since the standard deviation of daily changes for ADANIPOWER (2.536%) is higher than that of TATAPOWER (1.869%), ADANIPOWER had observed higher volatility in the last one month.

> Finished chain.


In [ ]:
response

{'input': 'Between tatapower and adanipower in NSE India, which stock had     observerd higher volatility in last one month? Explain the steps for your response.     The correct ticket symbol for NSE should be generated from the company names and passed as parameter to tools as necessary.',
 'output': 'To determine which stock had higher volatility in the last one month between TATAPOWER and ADANIPOWER, we need to compare their standard deviations of daily changes in percentage.\n\nThe standard deviation of daily changes in percentage for TATAPOWER is 1.782, while for ADANIPOWER it is 3.049. \n\nSince 3.049 is greater than 1.782, ADANIPOWER had higher volatility in the last one month.'}

In [26]:
import textwrap

In [27]:
wrapped_text = textwrap.fill(response['output'],
                             width=120,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Based on the stock trading data for the last one month, TATAPOWER had an average daily change of 0.39% and a standard
deviation of daily changes of 1.869%, while ADANIPOWER had an average daily change of -0.114% and a standard deviation
of daily changes of 2.536%. 

Since the standard deviation of daily changes for ADANIPOWER (2.536%) is higher than that
of TATAPOWER (1.869%), ADANIPOWER had observed higher volatility in the last one month.


## Exercise 1

Instruction: Add a tool (function), which can take investment amount and return percentage and return the final amount

for example:

```
get_total_return(principal: int, gain: float)
```

So, that we can ask questions like


**How much total return would been if I had invested 1000 rupees in the tatapower stock for last 3 months?**

The above prompt should call both the tools to first find out return for last 3 months and then using that calculate total return.